# Preprocessing

## Official Preprocessing Code for PT-BR Articles

In [ ]:
file_path = "data/processed/labeled_january_data.csv"

with open(file_path, "r", encoding="utf-8") as file:
    df_jan = pd.read_csv(file)

In [ ]:
file_path = "data/processed/labeled_february_data.csv"

with open(file_path, "r", encoding="utf-8") as file:
    df_feb = pd.read_csv(file)

In [ ]:
import spacy

# spacy PT model
nlp = spacy.load('pt_core_news_sm')

#preprocessing
def preprocess_text_spacy(text):
    doc = nlp(text)
    
    # lemmatization and stopwords removal
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    
    #tokens back to 1 string
    return ' '.join(tokens)

# preprocess ALL articles in january's dataframe
df_jan['preprocessed_article'] = df_jan['article'].apply(preprocess_text_spacy)
df_feb['preprocessed_article'] = df_feb['article'].apply(preprocess_text_spacy)


### Preprocessing January BDM Articles

In [ ]:
# def clean_file(inputFile, outputFile):
#     with open(inputFile, 'r', encoding='utf-8') as file:
#         cleaned_lines = []
        
#         for line in file:
            
#             cleaned_line = line.replace('[', '').replace(']', '').replace('…', '').strip()
            
#             if cleaned_line and cleaned_line[2] == '/' and cleaned_line[5] == '/':
#                 if cleaned_lines: 
#                     cleaned_lines.append('')  

#             if cleaned_line:
#                 cleaned_lines.append(cleaned_line)


#     with open(outputFile, 'w', encoding='utf-8') as file:
#         for line in cleaned_lines:
#             file.write(line + '\n')

# clean_file('data/news_corpus.txt', 'data/news_corpus_cleaned.txt')

## Organizing article file data into a DataFrame for readability and easier to convert to CSV
- PLEASE SAVE THIS OUTPUTTED DATA AS A CSV
- this is what i manually did for january, not exactly reproducable, be careful with this, i built the dataframe for february differently
- i need to get every data source into this dataframe format because it's easy to work with

In [ ]:
# FINSIHED PREPROCESSING JANUARY DATA, SAVED AS labeled_january_data.csv now, keeping this code for reusability later on

# import pandas as pd
# import re
# from datetime import datetime

# # look at the file and make it into a list
# def parse_articles_to_df(file_path):
#     dates = []  # keep the dates
#     articles = []  # keep the articles
    
#     with open(file_path, "r", encoding="utf-8") as file:
#         current_date = None  # keep the date we are looking at right now
#         current_articles = []  # keep the article for the current date
        
#         for line in file:
#             line = line.strip()  # take away spaces from the start and end
#             if line:  # if the line is not empty
#                 if line[2] == "/":  # if the line looks like a date (dd/mm/yy)
#                     # we had a date before, so let's save it with its articles
#                     if current_date:
#                         for article in current_articles:
#                             dates.append(current_date)  # add current date
#                             articles.append(article)  # add article for that date
#                     # use the date as is, don't change it
#                     current_date = line  # keep new date
#                     current_articles = []  # start fresh for new date
#                 else:
#                     # 'line' is the article, keep adding them to the list
#                     current_articles.append(line) 
        
#         if current_date:  # when we are done looking at the file
#             for article in current_articles:  # for all the articles we saved
#                 dates.append(current_date)  # add the date again
#                 articles.append(article)  # add the article again
    
#     df = pd.DataFrame({'date': dates, 'article': articles})  # make table with the dates and articles
#     return df  # display table

# file_path = "data/news_corpus_cleaned.txt"  # where the file is
# df_template = parse_articles_to_df(file_path)  # call function to make the table, called df_template because it can be used for another month

# # show the table to see it
# display(df_template.head())

In [ ]:
# checking what dates are in our file filled with articles

import re
import pandas as pd

def extract_dates_from_file(file_path):
    date_pattern = r'\d{2}/\d{2}/\d{2}'
    dates = []
    
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        matches = re.findall(date_pattern, content)
        dates.extend(matches)
    
    dates_df = pd.DataFrame(dates, columns=['Date'])
    return dates_df

file_path = 'data/news_corpus_cleaned.txt'
dates_df = extract_dates_from_file(file_path)
display(dates_df)

### Preprocessing February BDM Articles

In [ ]:
# import numpy as np
# import pandas as pd
# import os

# excel_file_path = "data/raw/bdm_march_clean_sort_of.xlsx"  
# output_directory = "data/processed/march_2024_csv_files"  

# os.makedirs(output_directory, exist_ok=True)
# excel_data = pd.ExcelFile(excel_file_path)  # load Excel file

# for sheet_name in excel_data.sheet_names:
#     # load current sheet into a DataFrame
#     df_sheet = excel_data.parse(sheet_name)
    
#     # construct output CSV file path
#     output_csv_path = os.path.join(output_directory, f"{sheet_name}.csv")
    
#     df_sheet.to_csv(output_csv_path, index=False, encoding="utf-8")
#     print(f"Saved {sheet_name} to {output_csv_path}")

Checking for NA or inf values

In [ ]:
import numpy as np
import pandas as pd
import os

file_path = "data/february_2024_csv_files/Sheet1.csv"

with open(file_path, "r", encoding="utf-8") as file:
    df = pd.read_csv(file)

print("NA Values: ", df["Label"].isna().any())
print("Infinity values: ", np.isinf(df["Label"]).any())

In [ ]:
invalid_rows = df[df["Label"].isna() | np.isinf(df["Label"])]
print(invalid_rows)

### Convert all NA labels in every sheet to 0 and convert them to ints instead of floats
- good for reproducibility, probably will implement into a script that can be reused to clean and label incoming articles

- DO NOT RUN THIS CODE AS IT WILL CORRUPT THE FILES, it is raw, manual preprocessing.

In [35]:
import os
import pandas as pd
import numpy as np

directory = "data/february_2024_csv_files"

for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        df.replace([np.inf, -np.inf], np.nan, inplace=True)
        df.fillna(0, inplace=True)
        df["Label"] = df["Label"].astype(int)
        df.to_csv(filepath, index=False)

In [46]:
# for filename in os.listdir(directory):
#     if filename.endswith(".csv"):
#         filepath = os.path.join(directory, filename)
#         df = pd.read_csv(filepath)
#         df.rename(columns={df.columns[1]: 'article'}, inplace=True)

#         df.to_csv(filepath, index=False)

In [54]:
# #sample of each sheet (can be used prior to preprocessing to check for missing values)

# file_path = "data/february_2024_csv_files/Sheet1.csv"

# with open(file_path, "r", encoding="utf-8") as file:
#     df = pd.read_csv(file)

# display(df)

,date,article,label
0,22/02/2024,… Mesmo reconhecendo que há uma desinflação si...,0
1,22/02/2024,"… Em uma 3ªF bastante movimentada, Haddad conv...",0
2,22/02/2024,… O ministro disse que o governo deverá enviar...,1
3,22/02/2024,… Isso significa que haveria um prazo de 45 di...,1
4,22/02/2024,"… Segundo Haddad, a sugestão para que o tema d...",1
...,...,...,...
81,22/02/2024,HYPERA. Assembleia aprovou aumento de capital ...,0
82,22/02/2024,TENDA informou que foi liquidada a operação re...,0
83,22/02/2024,… Montante total líquido recebido pela companh...,0
84,22/02/2024,"AERIS registrou prejuízo líquido de R$ 63,853 ...",0


In [61]:
# import pandas as pd

# # Read the problematic CSV

# file = "data/february_2024_csv_files/Sheet4.csv"
# df = pd.read_csv(file)

# # Fix the date column format
# df['date'] = pd.to_datetime(df['date']).dt.strftime('%m/%d/%y')

# # Save the updated CSV
# df.to_csv(file, index=False)

In [62]:
# import re

# all_data = []

# # sort filenames based on the numeric part of the filename (Sheet1, Sheet2, etc)
# filenames = sorted([filename for filename in os.listdir(directory) if filename.endswith(".csv")],
#                    key=lambda x: int(re.search(r'\d+', x).group()))

# for filename in filenames:
#     filepath = os.path.join(directory, filename)
#     df = pd.read_csv(filepath)
#     all_data.append(df)

# labeled_february_data_df = pd.concat(all_data, ignore_index=True)
# labeled_february_data_df.to_csv("labeled_february_data.csv", index=False)

# display(labeled_february_data_df)

### Rearrange CSVs with {date} and label columns to have date, article, and label columns and handles missing values

In [13]:
import os
import pandas as pd
import numpy as np

directory = "data/processed/march_news_corpus_cleaned"
dataframes = []
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        date = df.columns[0]
        df.rename(columns={date: 'article'}, inplace=True)
        df.insert(loc=0, column='date', value=date)
        dataframes.append(df)

final_df = pd.concat(dataframes, ignore_index=True)
final_df['label'] = final_df['label'].fillna(0)
final_df['label'] = final_df['label'].astype(int)
final_df.to_csv('data/processed/labeled_march_data.csv', index=False)

Merging files script

In [ ]:
#For Eli

import os

#check working directory 
os.getcwd() 

directory = os.listdir('data/News_Sample/andre')

for item in range(len(directory)):
    directory[item] = r'data/News_Sample/andre/' + directory[item]

#check for correct file names
print(directory)

#merge all files
with open('data/merged_files.txt', 'w') as outfile:
    for file in directory:
        fhand = open(file, 'r', encoding='utf-8')
        fname = fhand.read()
        outfile.write(fname)   

## Detailed Corpus Preprocessing for Lexicon Generation